In [ ]:
import asyncio
import random
from pyignite.datatypes.primitive_objects import IntObject
from pyignite.datatypes.standard import String
from perper import *

In [ ]:
task_collection = initialize_notebook()

In [ ]:
def do_something(message):
    print("DoSomething called:", message)


task_collection.add(listen_call_triggers(task_collection, "DoSomething", do_something))


async def do_something_async(message):
    print("DoSomethingAsync called:", message)


task_collection.add(listen_call_triggers(task_collection, "DoSomethingAsync", do_something))

In [ ]:
def get_random_number(a, b):
    return (random.randint(a, b), IntObject)


task_collection.add(listen_call_triggers(task_collection, "GetRandomNumber", get_random_number))


async def get_random_number_async(a, b):
    return (random.randint(a, b), IntObject)


task_collection.add(listen_call_triggers(task_collection, "GetRandomNumberAsync", get_random_number_async))

In [ ]:
randomNumber = await call_function("GetRandomNumber", [1, 100])
print(f"Random number: {randomNumber}")

anotherRandomNumber = await call_function("GetRandomNumberAsync", [1, 100])
print(f"Random number: {anotherRandomNumber}")

In [ ]:
await call_action("DoSomething", ["123"])
await call_action("DoSomethingAsync", ["456"])

In [ ]:
async def generator(count):
    for i in range(count):
        await asyncio.sleep(0.1)
        yield f"{i}. Message"


task_collection.add(listen_stream_triggers(task_collection, "Generator", generator))

In [ ]:
async def processor(generator, batch_size):
    batch = []
    async for message in stream_enumerate(generator):
        batch += [message + "_processed"]
        if len(batch) == batch_size:
            yield batch
            batch = []


task_collection.add(listen_stream_triggers(task_collection, "Processor", processor))

In [ ]:
async def consumer(processor):
    async for batch in stream_enumerate(processor):
        print(f"Received batch of {len(batch)} messages.")
        print(", ".join(batch))


task_collection.add(listen_stream_triggers(task_collection, "Consumer", consumer))

In [ ]:
message_count = 28
batch_count = 10

generator = create_stream_function("Generator", [message_count])
processor = create_stream_function("Processor", [generator, batch_count])
_ = create_stream_action("Consumer", [processor])